In [1]:
from typing import Dict, Tuple, Union
import numpy as np
import pulp


```js
Z->X->Y
Z<->X, X<->Y
```
All variables binary (0,1)

query: $Q = P(Y=1|do(X=1))$



### Compute bounds on $Q$ given $P(Z,X,Y)$ 

In [2]:
def bp_bounds_doX1_from_obs(p: Union[Dict[Tuple[int, int, int], float], np.ndarray],
                            tol: float = 1e-9) -> Tuple[float, float]:
    """
    Compute Balke–Pearl-style bounds for P(Y=1 | do(X=1)) in the binary DAG:
        Z -> X -> Y, with unobserved confounding on (Z,X) and (X,Y).

    Parameters
    ----------
    p : dict or np.ndarray
        Observational joint distribution p[z,x,y] = P(Z=z, X=x, Y=y).
        - If dict: keys are (z,x,y) with z,x,y in {0,1}.
        - If ndarray: shape must be (2,2,2) with axes in order (Z,X,Y).
    tol : float
        Tolerance for probability checks.

    Returns
    -------
    (lower, upper) : Tuple[float, float]
        Lower and upper bounds on P(Y=1 | do(X=1)).
    """
    # --- normalize/validate input ---
    if isinstance(p, dict):
        arr = np.zeros((2, 2, 2), dtype=float)
        for (z, x, y), prob in p.items():
            if z not in (0, 1) or x not in (0, 1) or y not in (0, 1):
                raise ValueError("dict keys must be (z,x,y) with each in {0,1}.")
            arr[z, x, y] = float(prob)
    else:
        arr = np.asarray(p, dtype=float)
        if arr.shape != (2, 2, 2):
            raise ValueError("ndarray input must have shape (2,2,2) with axes (Z,X,Y).")

    if np.any(arr < -tol):
        raise ValueError("Probabilities must be nonnegative.")
    total = arr.sum()
    if abs(total - 1.0) > 1e-6:
        # allow tiny drift; otherwise normalize (optional)
        if 1 - tol <= total <= 1 + tol:
            arr /= total
        else:
            raise ValueError(f"Joint distribution must sum to 1 (got {total}).")

    # Convenience aliases
    p_zxy = arr

    # --- Enumerate deterministic response-function types ---
    # Z has no parents: z_type ∈ {0,1}
    z_types = [0, 1]

    # X has parent Z: x_type is a boolean function of Z => pairs (x0, x1)
    bool_pairs = [(0, 0), (0, 1), (1, 0), (1, 1)]
    x_types = bool_pairs  # (x0, x1)

    # Y has parent X: y_type is a boolean function of X => pairs (y0, y1)
    y_types = bool_pairs  # (y0, y1)

    # Create an index over all types
    types = []
    for a in z_types:
        for b0, b1 in x_types:
            for c0, c1 in y_types:
                types.append((a, (b0, b1), (c0, c1)))
    assert len(types) == 32

    def _solve(optimize: str) -> float:
        """
        Solve LP to minimize or maximize P(Y=1 | do(X=1)).
        optimize ∈ {"min", "max"}
        """
        # LP
        prob = pulp.LpProblem("BP_bounds_doX1", pulp.LpMinimize if optimize == "min" else pulp.LpMaximize)

        # Decision variables: q[a,b,c] >= 0, sum q = 1
        q_vars = {}
        for idx, (a, (b0, b1), (c0, c1)) in enumerate(types):
            q_vars[idx] = pulp.LpVariable(f"q_{idx}", lowBound=0)

        # Normalization
        prob += pulp.lpSum(q_vars.values()) == 1, "normalization"

        # Observational fit: for each (z,x,y), sum of types consistent with that triple equals p_zxy[z,x,y]
        for z in (0, 1):
            for x in (0, 1):
                for y in (0, 1):
                    consistent = []
                    for idx, (a, (b0, b1), (c0, c1)) in enumerate(types):
                        x_z = b0 if z == 0 else b1
                        y_x = c0 if x == 0 else c1
                        if (a == z) and (x_z == x) and (y_x == y):
                            consistent.append(q_vars[idx])
                    # sum of consistent q's equals observed probability
                    prob += pulp.lpSum(consistent) == float(p_zxy[z, x, y]), f"obs_{z}{x}{y}"

        # Objective: P(Y=1 | do(X=1)) = sum_{types with y1=1} q
        objective_terms = []
        for idx, (a, (b0, b1), (c0, c1)) in enumerate(types):
            if c1 == 1:  # y1 = 1
                objective_terms.append(q_vars[idx])
        prob += pulp.lpSum(objective_terms), "target"

        # Solve
        solver = pulp.PULP_CBC_CMD(msg=False)
        status = prob.solve(solver)

        if pulp.LpStatus[status] != "Optimal":
            raise RuntimeError(f"LP did not solve to optimality (status={pulp.LpStatus[status]}).")

        value = pulp.value(prob.objective)
        # Clip tiny numerical drift
        if value < -1e-9 or value > 1 + 1e-9:
            raise RuntimeError(f"Objective outside [0,1] due to numerical issues: {value}")
        return max(0.0, min(1.0, value))

    lower = _solve("min")
    upper = _solve("max")
    return lower, upper

In [8]:
# Toy observational joint (must sum to 1). Axes order: (Z,X,Y).
# Here: close to independence just as a sanity check.
p = np.array([
    [[0.10, 0.10], [0.10, 0.10]],  # Z=0: X=0,Y=0/1 ; X=1,Y=0/1
    [[0.15, 0.15], [0.15, 0.15]],  # Z=1: ...
])
lo, hi = bp_bounds_doX1_from_obs(p)
print(f"P(Y=1 | do(X=1)) ∈ [{lo:.4f}, {hi:.4f}]")

P(Y=1 | do(X=1)) ∈ [0.2500, 0.7500]


### Compute bounds on $Q$ given $P(Z,X,Y)$ and $P(X,Y|do(Z))$

In [4]:
def bp_bounds_doX1_with_doZ(
    p: Union[Dict[Tuple[int, int, int], float], np.ndarray],
    z: Union[Dict[Tuple[int, int, int], float], np.ndarray],
    tol: float = 1e-9,
) -> Tuple[float, float]:
    """
    Balke–Pearl bounds for P(Y=1 | do(X=1)) in DAG:
        Z -> X -> Y with latent confounding on (Z,X) and (X,Y),
    using both observational P(Z,X,Y) and experimental P(X,Y | do(Z)).

    Parameters
    ----------
    p : dict or np.ndarray, shape (2,2,2), axes (Z,X,Y)
        Observational joint probabilities p[z,x,y] = P(Z=z, X=x, Y=y).
    z : dict or np.ndarray, shape (2,2,2), axes (Z_do, X, Y)
        Experimental tables z[z,x,y] = P(X=x, Y=y | do(Z=z)) for z∈{0,1}.
        For each fixed z, sum_{x,y} z[z,x,y] must be 1.
    tol : float
        Tolerance for probability checks.

    Returns
    -------
    (lower, upper) : Tuple[float, float]
        Lower and upper bounds on P(Y=1 | do(X=1)) under all SCMs
        consistent with both p and z.
    """
    # --- normalize/validate p ---
    if isinstance(p, dict):
        p_arr = np.zeros((2, 2, 2), dtype=float)
        for (zz, xx, yy), prob in p.items():
            if zz not in (0, 1) or xx not in (0, 1) or yy not in (0, 1):
                raise ValueError("p dict keys must be (z,x,y) with each in {0,1}.")
            p_arr[zz, xx, yy] = float(prob)
    else:
        p_arr = np.asarray(p, dtype=float)
        if p_arr.shape != (2, 2, 2):
            raise ValueError("p must have shape (2,2,2) with axes (Z,X,Y).")

    if np.any(p_arr < -tol):
        raise ValueError("p has negative entries.")
    p_total = p_arr.sum()
    if abs(p_total - 1.0) > 1e-6:
        if 1 - tol <= p_total <= 1 + tol:
            p_arr /= p_total
        else:
            raise ValueError(f"p must sum to 1 (got {p_total}).")

    # --- normalize/validate z ---
    if isinstance(z, dict):
        z_arr = np.zeros((2, 2, 2), dtype=float)
        for (zz, xx, yy), prob in z.items():
            if zz not in (0, 1) or xx not in (0, 1) or yy not in (0, 1):
                raise ValueError("z dict keys must be (z_do,x,y) with each in {0,1}.")
            z_arr[zz, xx, yy] = float(prob)
    else:
        z_arr = np.asarray(z, dtype=float)
        if z_arr.shape != (2, 2, 2):
            raise ValueError("z must have shape (2,2,2) with axes (Z_do,X,Y).")

    if np.any(z_arr < -tol):
        raise ValueError("z has negative entries.")
    for zz in (0, 1):
        s = float(z_arr[zz].sum())
        if abs(s - 1.0) > 1e-6:
            if 1 - tol <= s <= 1 + tol:
                z_arr[zz] /= s  # normalize that slice
            else:
                raise ValueError(f"z[{zz},:,:] must sum to 1 (got {s}).")

    # Enumerate response-function types
    z_types = [0, 1]  # Z has no parents
    bool_pairs = [(0, 0), (0, 1), (1, 0), (1, 1)]
    x_types = bool_pairs  # (x0, x1)
    y_types = bool_pairs  # (y0, y1)

    types = []
    for a in z_types:
        for b0, b1 in x_types:
            for c0, c1 in y_types:
                types.append((a, (b0, b1), (c0, c1)))
    assert len(types) == 32

    def _solve(optimize: str) -> float:
        prob = pulp.LpProblem("BP_bounds_doX1_with_doZ", pulp.LpMinimize if optimize == "min" else pulp.LpMaximize)

        # Decision variables λ (q_vars)
        q_vars = {idx: pulp.LpVariable(f"q_{idx}", lowBound=0) for idx in range(len(types))}
        prob += pulp.lpSum(q_vars.values()) == 1, "normalization"

        # Observational constraints: P(Z=z, X=x, Y=y)
        for z0 in (0, 1):
            for x in (0, 1):
                for y in (0, 1):
                    consistent = []
                    for idx, (a, (b0, b1), (c0, c1)) in enumerate(types):
                        x_z = b0 if z0 == 0 else b1
                        y_x = c0 if x == 0 else c1
                        if (a == z0) and (x_z == x) and (y_x == y):
                            consistent.append(q_vars[idx])
                    prob += pulp.lpSum(consistent) == float(p_arr[z0, x, y]), f"obs_{z0}{x}{y}"

        # Interventional constraints: P(X=x, Y=y | do(Z=z_do))
        # Under do(Z=z_do), Z is fixed; X = x_{z_do}, Y = y_{X}.
        for z_do in (0, 1):
            for x in (0, 1):
                for y in (0, 1):
                    consistent_do = []
                    for idx, (_a, (b0, b1), (c0, c1)) in enumerate(types):
                        x_z = b0 if z_do == 0 else b1
                        if x_z != x:
                            continue
                        y_x = c0 if x == 0 else c1
                        if y_x != y:
                            continue
                        consistent_do.append(q_vars[idx])
                    prob += pulp.lpSum(consistent_do) == float(z_arr[z_do, x, y]), f"doZ_{z_do}{x}{y}"

        # Objective: P(Y=1 | do(X=1)) = sum_{types with y1=1} λ
        objective_terms = []
        for idx, (_a, (_b0, _b1), (c0, c1)) in enumerate(types):
            if c1 == 1:
                objective_terms.append(q_vars[idx])
        prob += pulp.lpSum(objective_terms), "target"

        status = prob.solve(pulp.PULP_CBC_CMD(msg=False))
        st = pulp.LpStatus[status]
        if st != "Optimal":
            raise RuntimeError(f"LP status {st} (data may be incompatible).")
        val = pulp.value(prob.objective)
        return max(0.0, min(1.0, val))

    lower = _solve("min")
    upper = _solve("max")
    return lower, upper


In [9]:
# Experimental P(X,Y | do(Z=z))  (axes Z_do,X,Y), each slice sums to 1
z = np.zeros((2, 2, 2))
# Example slices (toy numbers)
z[0] = np.array([[0.30, 0.10],
                    [0.20, 0.40]])  # sums to 1
z[1] = np.array([[0.25, 0.25],
                    [0.25, 0.25]])  # sums to 1

lo, hi = bp_bounds_doX1_with_doZ(p, z)
print(f"Tightened bounds with do(Z): P(Y=1 | do(X=1)) ∈ [{lo:.4f}, {hi:.4f}]")

Tightened bounds with do(Z): P(Y=1 | do(X=1)) ∈ [0.4000, 0.7500]
